In [3]:
pip install streamlit

   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------  9.7/9.9 MB 54.9 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 44.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 31.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 47.4 MB/s eta 0:00:00

   -------- ------------------------------- 1/5 [narwhals]
   -------- ------------------------------- 1/5 [narwhals]
   -------- ------------------------------- 1/5 [narwhals]
   -------- ------------------------------- 1/5 [narwhals]
   ---------------- ----------------------- 2/5 [pydeck]
   ---------------- ----------------------- 2/5 [pydeck]
   ------------------------ --------------- 3/5 [altair]
   ------------------------ --------------- 3/5 [altair]
   ------------

In [12]:
# app.py — Melbourne House Price Predictor (clean Task 3)
import pandas as pd
import numpy as np
import streamlit as st
from pycaret.regression import load_model, predict_model

st.set_page_config(page_title="Melbourne House Price Predictor", page_icon="🏠", layout="centered")

@st.cache_resource(show_spinner=False)
def get_model():
    # PyCaret save_model(final_model, "models/melbourne_price_pipeline") -> load by base path
    return load_model("models/melbourne_price_pipeline")

model = get_model()

st.title("🏠 Melbourne House Price Predictor")
st.caption("Real-time predictions using a PyCaret pipeline (logged in MLflow)")

with st.expander("ℹ️ About", expanded=False):
    st.markdown(
        "- This app uses the same preprocessing as training (imputation, encoding, scaling).\n"
        "- Only runtime-available features are used (no leakage columns).\n"
    )

# ---------------- Sidebar inputs (runtime features only) ----------------
st.sidebar.header("Property Inputs")

Rooms = st.sidebar.number_input("Rooms", min_value=0, value=3, step=1)
Bedroom2 = st.sidebar.number_input("Bedroom2 (scraped)", min_value=0, value=3, step=1)
Bathroom = st.sidebar.number_input("Bathroom", min_value=0, value=2, step=1)
Car = st.sidebar.number_input("Car Spaces", min_value=0, value=1, step=1)

Distance = st.sidebar.number_input("Distance to CBD (km)", min_value=0.0, value=10.0, step=0.1)
Landsize = st.sidebar.number_input("Landsize (sqm)", min_value=0.0, value=450.0, step=10.0)
BuildingArea = st.sidebar.number_input("Building Area (sqm)", min_value=0.0, value=120.0, step=5.0)

YearBuilt = st.sidebar.number_input("Year Built", min_value=1800, max_value=2025, value=1998, step=1)
Propertycount = st.sidebar.number_input("Propertycount (in suburb)", min_value=0, value=6000, step=50)

SaleYear = st.sidebar.number_input("Sale Year", min_value=2007, max_value=2025, value=2017, step=1)
SaleMonth = st.sidebar.slider("Sale Month", min_value=1, max_value=12, value=6)

Method = st.sidebar.selectbox("Sale Method", options=["S", "Other"], index=0)
Type = st.sidebar.selectbox("Property Type", options=['h', 'u', 't', 'dev site', 'o res'], index=0)

Region = st.sidebar.text_input("Region (or 'Other')", value="Other")
CouncilArea = st.sidebar.text_input("Council Area (or 'Other')", value="Other")
Suburb = st.sidebar.text_input("Suburb (or 'Other')", value="Other")

use_location = st.sidebar.checkbox("Provide Latitude/Longitude?", value=False)
if use_location:
    Latitude = st.sidebar.number_input("Latitude", value=-37.80, step=0.01, format="%.5f")
    Longitude = st.sidebar.number_input("Longitude", value=145.00, step=0.01, format="%.5f")
else:
    Latitude = -37.80
    Longitude = 145.00

def build_input_df():
    property_age = max(0, int(SaleYear) - int(YearBuilt)) if YearBuilt > 0 else 0
    row = {
        'Rooms': int(Rooms),
        'Bedroom2': int(Bedroom2),
        'Bathroom': int(Bathroom),
        'Car': int(Car),
        'Distance': float(Distance),
        'Landsize': float(Landsize),
        'BuildingArea': float(BuildingArea),
        'YearBuilt': float(YearBuilt),
        'CouncilArea': CouncilArea.strip() or 'Other',
        'Region': Region.strip() or 'Other',
        'Suburb': Suburb.strip() or 'Other',
        'Method': Method,
        'Type': Type,
        'Propertycount': int(Propertycount),
        'SaleYear': int(SaleYear),
        'SaleMonth': int(SaleMonth),
        'PropertyAge': int(property_age),
        'Latitude': float(Latitude),
        'Longitude': float(Longitude),
    }
    return pd.DataFrame([row])

st.subheader("Enter details on the left, then click Predict")

col1, col2 = st.columns(2)
with col1:
    if st.button("Predict Price", type="primary", use_container_width=True):
        input_df = build_input_df()
        try:
            preds = predict_model(model, data=input_df)

            # robustly find prediction column (PyCaret 2.x vs 3.x)
            added_cols = [c for c in preds.columns if c not in input_df.columns]
            preferred = ['Label', 'prediction_label', 'Prediction', 'Predicted', 'Score']
            pred_col = next((c for c in preferred if c in preds.columns), None) or (added_cols[0] if added_cols else None)
            if pred_col is None:
                raise ValueError(f"Could not find prediction column. Columns: {list(preds.columns)}")

            price = float(preds[pred_col].iloc[0])
            st.success(f"💰 **Predicted Price:** ${price:,.0f}")

            with st.expander("See input & output (debug)"):
                st.write("Prediction column used:", pred_col)
                st.dataframe(preds, use_container_width=True)

        except Exception as e:
            st.error(f"Prediction failed: {e}")

with col2:
    st.info("This app uses a saved PyCaret pipeline (transformers + model). "
            "Change inputs to see the effect on price predictions.")

st.markdown("---")
st.caption("Model: `models/melbourne_price_pipeline.pkl` • Built with PyCaret • MLflow-tracked")

2025-08-27 00:27:07.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.236 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Transformation Pipeline and Model Successfully Loaded


2025-08-27 00:27:08.346 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 00:27:08.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [13]:
!jupyter nbconvert --to script Task3.ipynb --output app.py

[NbConvertApp] Converting notebook Task3.ipynb to script
[NbConvertApp] Writing 6595 bytes to app.py.py
